In [1]:
from langsmith import Client
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

import sys
from pathlib import Path

PROJECT_ROOT = Path("/Users/yuqiugan/Desktop/Personal_DS_Projects/house_rental_agent")
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.GoogleMaps_tools import query_google_maps
from src.Zillow_tools import submit_zillow_query_via_brightData, retrieve_snapshot_from_brightData
from src.Database_tools import bulk_upsert_listings
from src.Query_database_tool import agent_query_listing_table_tool
from src.settings import GOOGLE_GEMINI_API_KEY
from src.Helper_tools import get_date_time_now
from langchain_google_genai import ChatGoogleGenerativeAI

### HITL
from src.HITL_tools import request_human_approval

### Build Agent via LangChain

In [2]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GOOGLE_GEMINI_API_KEY,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

tools = [query_google_maps, submit_zillow_query_via_brightData, 
         retrieve_snapshot_from_brightData, bulk_upsert_listings, 
         agent_query_listing_table_tool, get_date_time_now, 
         request_human_approval]

system_prompt = """
You are a professional real estate broker. Use tools iteratively to gather *real* rental listings.

You have 7 tools:
- query_google_maps: query online transit route information from origin to destination.
- submit_zillow_query_via_brightData: submit a Zillow search to do online searching. Returns {{\"snapshot_id\": \"...\"}} on success.
- retrieve_snapshot_from_brightData: given snapshot_id, poll until data is ready (the tool handles waiting/retries internally). Returns listing data or {{\"status\": \"not_ready\"}} if it truly timed out. Though it need send online request, there is no cost, so you don't need human's approval for using this tool.
- bulk_upsert_listings: upsert the listing information to the local database listing table.
- agent_query_listing_table_tool: query listing information from local database, this will not cost any expense but may get some outdated listings. In the database's listing table, "updated_at" indicate the last time the record was updated,
  "created_at" indicates the first time that this record logged in the table. IMPORTANT: Pass spec as a JSON object, not a string or code block. Do not wrap in backticks.
- get_date_time_now: No input parameters, directly return the date time of now.
- request_human_approval: Before ANY online/network call, first call "request_human_approval" with a short action and concise details.
  Interpret the returned raw human text into one of: APPROVE / REJECT / MODIFY / DEFER / AMBIGUOUS.
  Rules:
    APPROVE: proceed with the original plan.
    REJECT: do not perform; summarize what would have been done.
    MODIFY: extract constraints (e.g., new budget/city/date) and proceed under those constraints.
    DEFER: stop and summarize when to resume.
    AMBIGUOUS: ask a clarifying question (one sentence), then wait.
  Always restate the interpreted decision and constraints in your reasoning (scratchpad) before acting.

CRITICAL EXECUTION RULES
- After calling submit_zillow_query_via_brightData, you MUST immediately call retrieve_snapshot_from_brightData with the snapshot_id returned.
- Submit exactly once per user request. If you already have a snapshot_id in this conversation, DO NOT call submit_zillow_query_via_brightData again. Wait until retrieve_snapshot_from_brightData get valid results.
- By default, if retrieve_snapshot_from_brightData return valid data, ALWAYS save to the local database for future reference.
- Do NOT produce a final answer until retrieve_snapshot_from_brightData returns real listing data (non-empty) or definitively reports no data (e.g., timed out after internal retries).
- If retrieve_snapshot_from_brightData returns {{\"status\":\"not_ready\"}} or similar, you may try it again. However, the tool itself already waits and retries; only re-call if it explicitly tells you it's safe to try again later.
- By default, when using query_google_maps to plan the route, the desination should always be user's request destination, the origin should be the address of the properties you got from retrieve_snapshot_from_brightData.
- By default, when user requests to check listings in a certain area, ALWAYS check local database first. If you find the records are out dated (no records in recent 5 days), search online then.
- When user ask about the details about the transit plan, alway generate transit map (set generate_route_map=True when calling query_google_maps)
- Keep your intermediate reasoning in the scratchpad; the user should only see the final JSON.


FINAL OUTPUT FORMAT (ONLY WHEN DONE)
Return a JSON object with:
- "Tool call history": String - brief tool call log.
- "Reasoning history": String - brief description of what you did and why.
- Return JSON only, do not write ``` json at the beginging or end of the JSON structure.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
        # ("placeholder", "{agent_scratchpad}")
        MessagesPlaceholder("agent_scratchpad")
    ]
)

_session_store = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in _session_store:
        _session_store[session_id] = ChatMessageHistory()
    return _session_store[session_id]


rental_agent = create_tool_calling_agent(llm, tools, prompt)
rental_agent_executor = AgentExecutor(
    agent = rental_agent, tools = tools, verbose=False, return_intermediate_steps=True
)

conversational_executor = RunnableWithMessageHistory(
    rental_agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

session_cfg = {"configurable": {"session_id": "yuqiu-local-dev"}}

In [3]:
query_message = "What's the date now?"
# response = rental_agent_executor.invoke({"input": query_message})
response = conversational_executor.invoke({"input": query_message}, config=session_cfg)

Key 'additionalProperties' is not supported in schema, ignoring


Key 'additionalProperties' is not supported in schema, ignoring


In [4]:
response.get('output')

'{"Tool call history": "default_api.get_date_time_now()", "Reasoning history": "The user asked for the current date, so I used the `get_date_time_now` tool to retrieve it.", "2025-09-10 00:35:13.968150"}'

In [5]:
output = response.get('output')
output2 = output.strip()

import re 
if output2.startswith("```"):
    output2 = re.sub(r"^```[a-zA-Z0-9]*\n", "", output2)
    output2 = re.sub(r"\n```$", "", output2)

# import json
# json.loads(output2)
output2

'{"Tool call history": "default_api.get_date_time_now()", "Reasoning history": "The user asked for the current date, so I used the `get_date_time_now` tool to retrieve it.", "2025-09-10 00:35:13.968150"}'

In [6]:
query_message = "Have you helped me before?"
# response = rental_agent_executor.invoke({"input": query_message})
response = conversational_executor.invoke({"input": query_message}, config=session_cfg)

Key 'additionalProperties' is not supported in schema, ignoring


In [7]:
output = response.get('output')
output2 = output.strip()

import re 
if output2.startswith("```"):
    output2 = re.sub(r"^```[a-zA-Z0-9]*\n", "", output2)
    output2 = re.sub(r"\n```$", "", output2)

# import json
# json.loads(output2)
output2

"As a large language model, I don't have memory of past conversations or personal interactions. Each interaction is fresh, and I don't retain any information from previous exchanges."

In [8]:
query_message = "My name is Yuqiu Gan."
# response = rental_agent_executor.invoke({"input": query_message})
response = conversational_executor.invoke({"input": query_message}, config=session_cfg)

Key 'additionalProperties' is not supported in schema, ignoring


In [9]:
output = response.get('output')
output2 = output.strip()

import re 
if output2.startswith("```"):
    output2 = re.sub(r"^```[a-zA-Z0-9]*\n", "", output2)
    output2 = re.sub(r"\n```$", "", output2)

# import json
# json.loads(output2)
output2

"It's nice to meet you, Yuqiu Gan! How can I help you today?"

In [10]:
query_message = "What is my name?"
# response = rental_agent_executor.invoke({"input": query_message})
response = conversational_executor.invoke({"input": query_message}, config=session_cfg)

output = response.get('output')
output2 = output.strip()

import re 
if output2.startswith("```"):
    output2 = re.sub(r"^```[a-zA-Z0-9]*\n", "", output2)
    output2 = re.sub(r"\n```$", "", output2)

# import json
# json.loads(output2)
output2

Key 'additionalProperties' is not supported in schema, ignoring


'Your name is Yuqiu Gan.'

In [11]:
query_message = "Which date is today?"
# response = rental_agent_executor.invoke({"input": query_message})
response = conversational_executor.invoke({"input": query_message}, config=session_cfg)

output = response.get('output')
output2 = output.strip()

import re 
if output2.startswith("```"):
    output2 = re.sub(r"^```[a-zA-Z0-9]*\n", "", output2)
    output2 = re.sub(r"\n```$", "", output2)

# import json
# json.loads(output2)
output2

Key 'additionalProperties' is not supported in schema, ignoring


Key 'additionalProperties' is not supported in schema, ignoring


'{"Tool call history": "default_api.get_date_time_now()", "Reasoning history": "The user asked for the current date, so I used the `get_date_time_now` tool to retrieve it.", "2025-09-10 00:36:06.122286"}'

In [12]:
query_message = "I want to find condos in weehawken, NJ area. My buget is $4500 and hope to find at least 2 bedrooms' unit. Can you list some available options?"
# response = rental_agent_executor.invoke({"input": query_message})
response = conversational_executor.invoke({"input": query_message}, config=session_cfg)

Key 'additionalProperties' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring


In [13]:
output = response.get('output')
output2 = output.strip()

import re 
if output2.startswith("```"):
    output2 = re.sub(r"^```[a-zA-Z0-9]*\n", "", output2)
    output2 = re.sub(r"\n```$", "", output2)

# import json
# json.loads(output2)
output2

'{"Tool call history": "default_api.agent_query_listing_table(spec = {\\"limit\\": 25, \\"order_by\\": [{\\"direction\\": \\"asc\\", \\"field\\": \\"listing_price\\"}], \\"select\\": [\\"external_id\\", \\"address_street\\", \\"address_city\\", \\"address_state\\", \\"bedrooms\\", \\"bathrooms\\", \\"listing_price\\", \\"home_type\\", \\"hdp_url\\", \\"updated_at\\"], \\"where\\": [{\\"field\\": \\"address_city\\", \\"op\\": \\"ilike\\", \\"value\\": \\"%Weehawken%\\"}, {\\"field\\": \\"address_state\\", \\"op\\": \\"=\\", \\"value\\": \\"NJ\\"}, {\\"field\\": \\"listing_price\\", \\"op\\": \\"<=\\", \\"value\\": 4500}, {\\"field\\": \\"bedrooms\\", \\"op\\": \\">=\\", \\"value\\": 2}, {\\"field\\": \\"home_type\\", \\"op\\": \\"ilike\\", \\"value\\": \\"%CONDO%\\"}]})\\ndefault_api.get_date_time_now()", "Reasoning history": "I searched the local database for condos in Weehawken, NJ, with at least 2 bedrooms and a maximum price of $4500. I found one listing that matches your criteria a

In [14]:
query_message = "Are there any more available options?"
# response = rental_agent_executor.invoke({"input": query_message})
response = conversational_executor.invoke({"input": query_message}, config=session_cfg)
output = response.get('output')
output2 = output.strip()

import re 
if output2.startswith("```"):
    output2 = re.sub(r"^```[a-zA-Z0-9]*\n", "", output2)
    output2 = re.sub(r"\n```$", "", output2)

output2

Key 'additionalProperties' is not supported in schema, ignoring


Key 'additionalProperties' is not supported in schema, ignoring



=== HUMAN APPROVAL REQUIRED ===


'{"Tool call history": "default_api.request_human_approval(action=\'The listings in the database might be outdated. Do you want me to search online for more recent condo listings in Weehawken, NJ, with at least 2 bedrooms and a budget of $4500?\')", "Reasoning history": "The user asked for more options. I have already provided one listing from the local database. To find more, I need to perform an online search. As per the instructions, I must request human approval before making any online calls. I have asked for approval to search online for more recent condo listings in Weehawken, NJ, with at least 2 bedrooms and a budget of $4500.", "question": "The listings in the database might be outdated. Do you want me to search online for more recent condo listings in Weehawken, NJ, with at least 2 bedrooms and a budget of $4500?", "status": "NEEDS_HUMAN"}'

In [15]:
query_message = "Sure, help me to search online."
response = conversational_executor.invoke({"input": query_message}, config=session_cfg)
output = response.get('output')
output2 = output.strip()

import re 
if output2.startswith("```"):
    output2 = re.sub(r"^```[a-zA-Z0-9]*\n", "", output2)
    output2 = re.sub(r"\n```$", "", output2)

output2

Key 'additionalProperties' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring


[1/3] Snapshot not ready, retrying in 60s…
[2/3] Snapshot not ready, retrying in 60s…
[3/3] Snapshot not ready, retrying in 60s…


Key 'additionalProperties' is not supported in schema, ignoring


[1/3] Snapshot not ready, retrying in 60s…
[2/3] Snapshot not ready, retrying in 60s…
[3/3] Snapshot not ready, retrying in 60s…


Key 'additionalProperties' is not supported in schema, ignoring


...

retrieved snapshot: s_mfdhwnoo1x8f53dztj


Key 'additionalProperties' is not supported in schema, ignoring


Calling bulk_upsert_listings function...


Key 'additionalProperties' is not supported in schema, ignoring


'{"Tool call history": "default_api.submit_zillow_query_via_brightData(HomeType=\'Condos\', days_on_zillow=\'7 days\', listingCategory=\'House for rent\', location=\'Weehawken, NJ\')\\ndefault_api.retrieve_snapshot_from_brightData(snapshot_id=\'s_mfdhwnoo1x8f53dztj\')\\ndefault_api.retrieve_snapshot_from_brightData(snapshot_id=\'s_mfdhwnoo1x8f53dztj\')\\ndefault_api.retrieve_snapshot_from_brightData(snapshot_id=\'s_mfdhwnoo1x8f53dztj\')\\ndefault_api.bulk_upsert_listings(snapshot_file_path=\'/Users/yuqiugan/Desktop/Personal_DS_Projects/house_rental_agent/data/s_mfdhwnoo1x8f53dztj.json\')\\ndefault_api.agent_query_listing_table(spec={\'limit\': 25, \'order_by\': [{\'direction\': \'asc\', \'field\': \'listing_price\'}], \'select\': [\'external_id\', \'address_street\', \'address_city\', \'address_state\', \'bedrooms\', \'bathrooms\', \'listing_price\', \'home_type\', \'hdp_url\', \'updated_at\'], \'where\': [{\'field\': \'address_city\', \'op\': \'ilike\', \'value\': \'%Weehawken%\'}, {\

In [16]:
query_message = "OK, in that case, my company is at '277 Park Ave, New York, NY 10172', if I move to this place how should I transit to my company?"
response = conversational_executor.invoke({"input": query_message}, config=session_cfg)
output = response.get('output')
output2 = output.strip()

import re 
if output2.startswith("```"):
    output2 = re.sub(r"^```[a-zA-Z0-9]*\n", "", output2)
    output2 = re.sub(r"\n```$", "", output2)

output2

Key 'additionalProperties' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring


'{"Tool call history": "default_api.query_google_maps(destination=\'277 Park Ave, New York, NY 10172\', generate_route_map=True, origin=\'1055 Boulevard E APT C11, Weehawken, NJ\', origin_name=\'Weehawken Condo\')", "Reasoning history": "The user asked for transit information from the Weehawken condo to their company. I used the `query_google_maps` tool to get the public transit route, setting `generate_route_map` to `True` to provide a visual aid.", "Here\'s the transit plan from 1055 Boulevard E APT C11, Weehawken, NJ to 277 Park Ave, New York, NY 10172:\\n\\n**Total Duration:** 45 minutes\\n**Total Distance:** 5.2 miles\\n\\n**Steps:**\\n1.  **Walk** to Boulevard East at 50th St.\\n2.  Take the **128 Bus** towards New York from Boulevard East at 50th St. (approx. 21 minutes, 10 stops) to Port Authority Bus Terminal.\\n3.  **Walk** to Times Sq–42 St.\\n4.  Take the **7 Train (Flushing Local and Express)** towards Flushing-Main St from Times Sq–42 St (approx. 3 minutes, 2 stops) to Gr

In [17]:
query_message = "I don't think that's the conda I want. Can you find listings in other area? Say, Edgewater NJ?"
response = conversational_executor.invoke({"input": query_message}, config=session_cfg)
output = response.get('output')
output2 = output.strip()

import re 
if output2.startswith("```"):
    output2 = re.sub(r"^```[a-zA-Z0-9]*\n", "", output2)
    output2 = re.sub(r"\n```$", "", output2)

output2

Key 'additionalProperties' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring


'{"Tool call history": "default_api.agent_query_listing_table(spec={\'limit\': 25, \'order_by\': [{\'direction\': \'asc\', \'field\': \'listing_price\'}], \'select\': [\'external_id\', \'address_street\', \'address_city\', \'address_state\', \'bedrooms\', \'bathrooms\', \'listing_price\', \'home_type\', \'hdp_url\', \'updated_at\'], \'where\': [{\'field\': \'address_city\', \'op\': \'ilike\', \'value\': \'%Edgewater%\'}, {\'field\': \'address_state\', \'op\': \'=\', \'value\': \'NJ\'}, {\'field\': \'listing_price\', \'op\': \'<=\', \'value\': 4500}, {\'field\': \'bedrooms\', \'op\': \'>=\', \'value\': 2}, {\'field\': \'home_type\', \'op\': \'ilike\', \'value\': \'%CONDO%\'}]})\\ndefault_api.request_human_approval(action=\'The local database does not have any recent listings for condos in Edgewater, NJ, that meet your criteria. Do you want me to search online for available options?\', details=\'Search for condos in Edgewater, NJ, with at least 2 bedrooms and a budget of $4500.\')", "Rea

In [18]:
query_message = "No, I don't need online searches."
response = conversational_executor.invoke({"input": query_message}, config=session_cfg)
output = response.get('output')
output2 = output.strip()

import re 
if output2.startswith("```"):
    output2 = re.sub(r"^```[a-zA-Z0-9]*\n", "", output2)
    output2 = re.sub(r"\n```$", "", output2)

output2

Key 'additionalProperties' is not supported in schema, ignoring


"Understood. Since you don't want to search online, and my local database does not have any condo listings in Edgewater, NJ, with at least 2 bedrooms and a budget of $4500, I cannot provide any options for that area at this time.\n\nWould you like to adjust your search criteria (e.g., different home type, fewer bedrooms, higher budget) or explore another location within the existing database?"

In [23]:
query_message = "I see, in that case, which apartment you think suit me best? I want to take both the convinience of transit and the price into consideration. Give me reasons."
response = conversational_executor.invoke({"input": query_message}, config=session_cfg)
output = response.get('output')
output2 = output.strip()

import re 
if output2.startswith("```"):
    output2 = re.sub(r"^```[a-zA-Z0-9]*\n", "", output2)
    output2 = re.sub(r"\n```$", "", output2)

output2

Key 'additionalProperties' is not supported in schema, ignoring


"Considering both transit convenience and price, I would recommend one of the **2-bedroom, 2-bathroom units at 1300 Avenue At Port, Weehawken, NJ (Riverparc at Port Imperial)**.\n\nHere's why:\n\n*   **Transit Convenience:** These units offer the shortest transit time to your company at 277 Park Ave, New York, NY 10172, at approximately **34 minutes**. This is a significant improvement compared to the other listings, which have commute times closer to 48-49 minutes. While it's still more than your desired 20 minutes, it's the most convenient option among the available listings.\n\n*   **Price:** While these units are at the higher end of your $4500 budget (starting from $4138), they still fall within your specified range. The other listings are more affordable, but the trade-off is a much longer commute.\n\n**Recommendation:**\n\nSpecifically, the unit at **1300 Avenue At Port, Weehawken, NJ** with an external ID of **2063201591** is priced at **$4138**. This unit offers the best balan

In [24]:
query_message = "I would love to live in a place that has natural lights. Is there any in your result?"
response = conversational_executor.invoke({"input": query_message}, config=session_cfg)
output = response.get('output')
output2 = output.strip()

import re 
if output2.startswith("```"):
    output2 = re.sub(r"^```[a-zA-Z0-9]*\n", "", output2)
    output2 = re.sub(r"\n```$", "", output2)

output2

Key 'additionalProperties' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring


'Yes, I found two listings that explicitly mention having natural light:\n\n1.  **4612 Park Ave #101A, Weehawken, NJ 07086**\n    *   The general description states: "Tons of sunlight Plenty of windows"\n\n2.  **61 46th St #2, Weehawken, NJ 07086**\n    *   The general description states: "Tons of natural light"\n\nFor the other listings, particularly those at 1300 Avenue At Port (Riverparc at Port Imperial), the detailed descriptions regarding natural light are not available in my current records.'

In [12]:
_session_store

{'yuqiu-local-dev': InMemoryChatMessageHistory(messages=[HumanMessage(content="What's the date now?", additional_kwargs={}, response_metadata={}), AIMessage(content='{"Tool call history": "Called get_date_time_now to get the current date and time.", "Reasoning history": "The user asked for the current date, so I used the `get_date_time_now` tool to retrieve it."}', additional_kwargs={}, response_metadata={}), HumanMessage(content='Have you helped me before?', additional_kwargs={}, response_metadata={}), AIMessage(content="As a large language model, I don't have memory of past conversations with you. Each interaction is fresh. How can I help you today?", additional_kwargs={}, response_metadata={}), HumanMessage(content='My name is Yuqiu Gan.', additional_kwargs={}, response_metadata={}), AIMessage(content="It's nice to meet you, Yuqiu Gan! How can I assist you today?", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is my name?', additional_kwargs={}, response_me

In [13]:
# 1) Send a name
conversational_executor.invoke({"input": "My name is Yuqiu Gan."}, config=session_cfg)

# 2) Ask what my name is
out = conversational_executor.invoke({"input": "What is my name?"}, config=session_cfg)
print(out["output"] if isinstance(out, dict) and "output" in out else out)


Key 'additionalProperties' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring


As a large language model, I don't have memory of past conversations, so I don't know your name.


### HITL via Langgraph